# BUILD OPTIMIZED MODEL
### (with data preparation!)

In [1]:
#r "nuget:Microsoft.ML,1.5.1"
using Microsoft.ML;
using Microsoft.ML.Data;

The below script needs to be able to find the current output cell; this is an easy method to get it.

Installed package Microsoft.ML version 1.5.1

#### Load data models from Models.cs file 

In [2]:
#load "C:\Users\dcost\source\repos\SmartFireAlarm\SmartFireAlarm\Jupyter\Models.csx"

#### Initialize the ML context (we need it for building the data and training pipelines)  

In [3]:
MLContext mlContext = new MLContext();

#### Load data from csv file into a dataview

In [4]:
const string DATASET_PATH = "./sensors_data.csv";
IDataView data = mlContext.Data.LoadFromTextFile<ModelInput>(
    path: DATASET_PATH,
    hasHeader: true,
    separatorChar: ',');

#### Shuffle and split data to _trainingData_ and _testingData_ by a fraction of 0.2

In [5]:
var shuffledData = mlContext.Data.ShuffleRows(data, seed: 0);
var split = mlContext.Data.TrainTestSplit(shuffledData, testFraction: 0.2);
var trainingData = split.TrainSet;
var testingData = split.TestSet;

#### Model builder pipeline

In [6]:
var featureColumns = new string[] { "Temperature", "Luminosity", "Infrared" };

var trainingPipeline = mlContext.Transforms.Conversion.MapValueToKey("Label")
    .Append(mlContext.Transforms.Concatenate("Features", featureColumns))
    .Append(mlContext.Transforms.NormalizeMinMax("Features"))
    .Append(mlContext.MulticlassClassification.Trainers.SdcaNonCalibrated("Label", "Features"))
    .Append(mlContext.Transforms.Conversion.MapKeyToValue("PredictedLabel"));


(3,24): warning CS1701: Assuming assembly reference 'Microsoft.AspNetCore.Html.Abstractions, Version=2.2.0.0, Culture=neutral, PublicKeyToken=adb9793829ddae60' used by 'Microsoft.DotNet.Interactive.Formatting' matches identity 'Microsoft.AspNetCore.Html.Abstractions, Version=3.1.7.0, Culture=neutral, PublicKeyToken=adb9793829ddae60' of 'Microsoft.AspNetCore.Html.Abstractions', you may need to supply runtime policy



#### Train the model

In [7]:
var model = trainingPipeline.Fit(trainingData);

#### Evaluate the model 

In [8]:
var predictions = model.Transform(testingData);
var metrics = mlContext.MulticlassClassification.Evaluate(predictions, "Label", "Score", "PredictedLabel");
display(p[@class:"large_label"]($"MicroAccuracy: {metrics.MicroAccuracy:P2}, MacroAccuracy: {metrics.MacroAccuracy:P2}"));

MicroAccuracy: 96.00%, MacroAccuracy: 96.84%

#### Generate a sample input

In [ ]:
var sampleData = new ModelInput
{
    Luminosity = 92F,
    Temperature = 22F,
    Infrared = 0F,
    Distance = 0F,
    CreatedAt = "01/03/2020 10:22:08"
};

#### Predict data

In [ ]:
var predictor = mlContext.Model.CreatePredictionEngine<ModelInput, ModelOutput>(model);
var predicted = predictor.Predict(sampleData);

display(p[@class:"very_large_label"]($"{predicted.PredictedLabel}"));